In [11]:
import pandas as pd
import time
from binance.client import Client
from datetime import datetime
import numpy as np
import json
from binance import ThreadedWebsocketManager
from binance.exceptions import BinanceAPIException, BinanceOrderException
import talib
class Binance_auto:
    api_key = 'a85aa4b4471a83d94169c40a9a93c39d9ebb508e44c5092fa6efd6253ae56449'
    api_secret = '599972b1fde19ed879f3dcfcc4ce83d858d21d2008ec332f18b0af0131f213bd'
    lag=-65
    base_ra=29
    con_ra=8
    spanB_value=30
    twm = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret,testnet=True)
    client = Client(api_key, api_secret)
    Daily_date=pd.DataFrame() 
    
    
    def __init__(self):
        pass
        # 클라이언트 생성, 즉 전체 함수 시작 전 사용됨.


    def Future_balance(self):
        client=self.client
        future_balance=client.futures_account_balance()
        
        if future_balance[6]['asset']=='USDT':

            future_balace=round(float(future_balance[6]['balance']),2)
            
            return future_balace
        return False
        # 계정 계좌 가져오는 함수. USDT만 float 형태로 round 반올림해서 가져옴. future_balance의 balance 가져옴. 


    def future_buy(self,present_price): # 롱 포지션
        client=self.client

        qtd_quote = self.Future_balance() # 잔액 불러와서 qtd_quote 변수에 넣음
        size = round(qtd_quote/present_price,3) # 잔액을 최신가격으로 나눔. 최대 구매할 수 있는 수가 즉 size. 반올림해서 소수점 3자리까지.
        # Trade=client.futures_account_trades()[-1]
        # put_money=float(Trade['quoteQty'])
        
        sl=float(round(present_price*0.981,2)) # 스탑로스 최신 가격에 1,9 % 곱한 것. 즉 2퍼센트 손해보면 손절. 소수점 2자리까지 반올림
        tp=float(round(present_price*0.008+present_price,2)) # 테이크 프로핏 즉 손익. 최신가격에 원하는 손익 %를 곱한 후 최신가격을 더한 값.
       
        try:
            client.futures_create_order(symbol='BTCUSDT', side='BUY', # futures_Create_order 라이브러리를 사용 심볼, 매매 선택, 거래방식(시장가, 지정가), 거래규모
        type='MARKET', quantity=size)
            time.sleep(1)
    
            client.futures_create_order(symbol='BTCUSDT', side='SELL',
        type='STOP_MARKET', stopPrice=f'{sl}',closePosition='true') # 나머지는 같음. stopPrice 즉 거래 그만 두는 것. 위에 지정한 스탑 로스에 도달하면 손절.
      
            time.sleep(1)
            client.futures_create_order(symbol='BTCUSDT', side='SELL',
            type='TAKE_PROFIT_MARKET', stopPrice=f'{tp}',closePosition='true') # 위와 같음 TP에 지정한 손익에 도달하면 거래 중단.
        

        except BinanceAPIException as e:
            print(e)
        except BinanceOrderException as e: # 오류 날 시 해당 class 에서 처리.
        # error handling goes here
            print(e)

#  self.sell(tp=.979*Binance_price.Close[-1],sl=Binance_price.Close[-1]*.0198+Binance_price.Close[-1])

    def future_sell(self,present_price): # 숏 포지션.
        client=self.client
        qtd_quote = self.Future_balance()
        size = round(qtd_quote/present_price,3)
        sl=(present_price*0.198,2)+present_price
        tp=(present_price*0.979,2)

        try:
            client.futures_create_order(symbol='BTCUSDT', side='BUY',
    positionSide = 'SHORT', type='MARKET', quantity=size)
            client.futures_create_order(symbol='BTCUSDT', side='SELL',
    type='STOP_MARKET', stopPrice=f'{sl}',closePosition='true')
            client.futures_create_order(symbol='BTCUSDT', side='SELL',
                type='TAKE_PROFIT_MARKET', stopPrice=f'{tp}',closePosition='true')
   
        except BinanceAPIException as e:
            print(e)
        except BinanceOrderException as e:
        # error handling goes here
            print(e)

    


    def Future_Hisorical_data(self,stocks,time,start): # 과겨 가격 데이터를 가져옴. 왜 쓰는지? 이동평균선 등 기존에 존재하는 지표를 구현하기 위해 필요로 함.
        client=self.client
        client.futures_historical_klines(stocks, time,  start) # futures historical klines 라이브러리, stock = 거래를 원하는 심볼 즉 BTCUSDT, time= 시간, start = 원하는 시간 대
    
    
        Price_data=pd.DataFrame(client.futures_historical_klines(stocks, time,  start)) # 가격 데이터를 이와 같이 판다스 데이터프레임화 시켜 변수 지정.
        # print(Price_data)
        
        Price_data=Price_data.iloc[:, :6] # 원래는 12가지의 컬럼이 있지만 columns를 사용하여 Date 부터 Volume 까지 지정.
        Price_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        Price_data=Price_data.set_index('Date') # set_index()함수는 데이터프레임을 먼저 불러온 후에, 특정 열을 인덱스로 설정할때 사용 즉 Datem를 인덱스로 설정.
        
        return Price_data

        # 과거 데이터 가져온 후 판다스 데이터 프레임화 시키고 컬럼 지정. 6개를 제외한 컬럼 무시.

    def handle_socket_message(self,msg): 
        if msg['e'] == 'error':
            # self.twm.stop()
            time.sleep(2) # 오류 발생 시 e 메시지 송신, 멈춤. 
            # self.main()
        else:
           # 오류 발생하지 않으면 아래 정보들 호출.
            client=self.client
            
            future_balance=client.futures_account_balance() # 선물 계좌 잔액. 변수 지정
            present_balance=self.Future_balance() # 최신 잔액. 변수 지정.

            Binance_price=self.Future_Hisorical_data('BTCUSDT','15m',"1 day ago UTC") # 과거 데이터를 가져온 것 에서 종목, 15분 봉, 하루 전 UTC 시간대 가져옴. (협정 시간대)
 
            Binance_price=Binance_price.astype(float) # 위에서 지정한 과거 데이터를 float형태로 변환.
            Binance_price.loc[int(msg['E'])]=[float(msg['k']['o']),float(msg['k']['h']),float(msg['k']['l']),float(msg['k']['c']),float(msg['k']['v'])]
            # 소켓을 통해 가져온 정보를 E 데이트 타임, k에 할당된 시초가,고가, 저가, 종가, 거래량 가져옴. 
            # 만약 하루 전 15분 봉 데이터라면 09시 기준 데이터가 프레임화 되었을때 09시 15분 데이터가 밑으로 들어감. 
            # 하루 전 데이터프레임이 완성되면 while 반복문 처럼 순회하여 다시 09시 부터 만들어짐
            # 심볼의 현재 가격 데이터가 계속 변하므로 위와 같은 과정이 필요함.
            postion_information=client.futures_position_information(symbol="BTCUSDT")[0]['entryPrice'] 
            # futures position information 라이브러리에 심볼 지정. 원하는 정보 호출.  #entryPrice : 평균 진입가 호출 후 position information 변수에 지정.
           
            # print('현재',Binance_price.Close.iloc[-1])
            

            time.sleep(1)
            


           
            if (postion_information=='0.0'):
               
         
                            self.future_buy(float(msg['k']['c'])) # 종가.




    def main(self):  # 위에 지정한 함수 불러와서 실행

        symbol ='BTCUSDT'
        self.Daily_date=self.Future_Hisorical_data('BTCUSDT','1d','1 Jan,2018') # 과거 데이터 호출 

        # start is required to initialise its internal loop
        self.twm.start() # thread websocket manager start 메서드 사용. 내부 루트 초기화?

        # twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)
        future_money=self.twm.start_kline_futures_socket(callback=self.handle_socket_message, symbol=symbol,interval='1m')
        # future_money 변수에 심볼 지정, 주기 지정. 위에 지정한 handle_socket_message 로 콜백 시킴.



        # self.twm.stop()
        self.twm.join()
        


        # 하나하나 다 실행해볼것

       
    


In [2]:
from binance.client import Client
import pandas as pd
api_key = 'a85aa4b4471a83d94169c40a9a93c39d9ebb508e44c5092fa6efd6253ae56449'
api_secret ='599972b1fde19ed879f3dcfcc4ce83d858d21d2008ec332f18b0af0131f213bd'
client = Client(api_key, api_secret, testnet=True)


In [3]:
def Future_balance(self):
        client=self.client
        future_balance=client.futures_account_balance()
        
        if future_balance[6]['asset']=='USDT':

            future_balace=round(float(future_balance[6]['balance']),2)
            
            return future_balace
        return False

In [14]:
df = pd.DataFrame(client.futures_account_balance())['balance'][3]

In [15]:
print(df)

9249.40935345


In [ ]:
dd ='e': 'continuous_kline', 'E': 1672292301485, 'ps': 'BTCUSDT', 'ct': 'PERPETUAL', 'k': {'t': 1672292280000, 'T': 1672292339999, 'i': '1m', 'f': 29087764789, 'L': 29087767457, 'o': '16570.50', 'c': '16646.90', 'h': '16646.90', 'l': '16570.50', 'v': '24.009', 'n': 8, 'x': False, 'q': '398481.40490', 'V': '8.086', 'Q': '134606.83340', 'B': '0'}